In [ ]:
%pip install datasets transformers accelerate -U transformers[torch] dask flash-attn --no-build-isolation

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.3/9.3 MB 43.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 30.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 11.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 2.1 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1

In [2]:
from datasets import load_dataset, load_metric
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC, TrainingArguments, Trainer, DataCollatorWithPadding
import torch
import dask.dataframe as dd
import numpy as np
import pandas as pd
import re

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [1]:
# dataset retirado do https://commonvoice.mozilla.org/pt/datasets
# common_voice_11_0 e
def dataset(linguagem:str):
    dataset_voice = load_dataset("mozilla-foundation/common_voice_11_0", linguagem, split="train+validation", trust_remote_code=True)
    test_dataset_voice = load_dataset("mozilla-foundation/common_voice_11_0", linguagem, split="test", trust_remote_code=True)

    return dataset_voice, test_dataset_voice

dataset_voice, test_dataset_voice = dataset("pt")

NameError: name 'load_dataset' is not defined

In [ ]:
dataset_voice[0]["audio"]

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

processor = Wav2Vec2Processor.from_pretrained("facebook/wav2vec2-large-960h", sampling_rate=48000) #facebook/wav2vec2-base-960h
model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-large-960h", #facebook/wav2vec2-base-960h
                                       ctc_loss_reduction="mean",
                                       torch_dtype=torch.float16,
                                       attn_implementation="flash_attention_2",
                                       pad_token_id=processor.tokenizer.pad_token_id).to(device)

In [ ]:
type(dataset_voice), type(test_dataset_voice)

In [ ]:
dataset_voice, test_dataset_voice

In [ ]:
dataset_voice_processed = dataset_voice.remove_columns(['client_id', 'accent', 'gender', 'age', 'up_votes', 'path', 'down_votes'])
test_dataset_voice_processed = test_dataset_voice.remove_columns(['client_id', 'accent', 'gender', 'age', 'up_votes', 'path', 'down_votes'])

In [ ]:
dataset_voice_processed, test_dataset_voice_processed

In [ ]:
def remove_special_characters(observation):
    chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"]'
    observation["sentence"] = re.sub(chars_to_ignore_regex, '', observation["sentence"]).lower() + " "
    return observation

dataset_voice_processed = dataset_voice_processed.map(remove_special_characters)
test_dataset_voice_processed = test_dataset_voice_processed.map(remove_special_characters)

In [ ]:
num_lines_to_exclude = 10000
total_lines = dataset_voice.num_rows
indices_to_keep = list(range(num_lines_to_exclude, total_lines))
filtered_train_dataset = dataset_voice.select(indices_to_keep)

In [ ]:
def prepare_dataset(observation):
    observation["input_ids"] = processor(observation["audio"]["array"], sampling_rate=observation["audio"]["sampling_rate"], return_tensors="pt").input_values[0]
    with processor.as_target_processor():
        observation["labels"] = processor(observation["sentence"]).input_ids
    return observation

dataset_voice_processed = dataset_voice_processed.map(prepare_dataset, remove_columns=dataset_voice_processed.column_names)
test_dataset_voice_processed = test_dataset_voice_processed.map(prepare_dataset, remove_columns=test_dataset_voice_processed.column_names)

In [ ]:
# Renomear a coluna 'input_values' para 'input_ids' no dataset de treinamento
#dataset_voice_processed = dataset_voice_processed.rename_column("input_values", "input_ids")

# Renomear a coluna 'input_values' para 'input_ids' no dataset de teste
#test_dataset_voice_processed = test_dataset_voice_processed.rename_column("input_values", "input_ids")

In [ ]:
dataset_voice_processed.column_names, test_dataset_voice_processed.column_names

In [ ]:
# Função para calcular as métricas
def compute_metrics(pred):
    wer_metric = load_metric("wer")
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)
    pred.label_ids[pred.label_ids == -100] = processor.tokenizer.pad_token_id
    pred_str = processor.batch_decode(pred_ids)
    label_str = processor.batch_decode(pred.label_ids, group_tokens=False)
    wer = wer_metric.compute(predictions=pred_str, references=label_str)
    return {"wer": wer}

In [ ]:
data_collator = DataCollatorWithPadding(processor.tokenizer)
data_collator

In [ ]:
# Configurar argumentos de treinamento
training_args = TrainingArguments(
  output_dir="./wav2vec2",
  per_device_train_batch_size=16,
  per_device_eval_batch_size=16,
  num_train_epochs=3,
  learning_rate=1e-4,
  weight_decay=0.01,
)

# Configurar o DataCollator
data_collator = DataCollatorWithPadding(processor.tokenizer)

# Configurar o Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset_voice_processed,
    eval_dataset=test_dataset_voice_processed,
    data_collator=data_collator,
    tokenizer=processor.feature_extractor,  # Certifique-se de que o tokenizador esteja correto
)

# Treinar o modelo
trainer.train()

In [ ]:
# Avaliar e salvar o modelo
trainer.evaluate(test_dataset)
model.save_pretrained("./wav2vec2")
processor.save_pretrained("./wav2vec2")